# MWS Secondary Target Selection

## MWS_CALIB_APOGEE

Maintainers:
  - Ting Li (selection/catalog)
  - Andrew Cooper (secondary target code)

In [19]:
!pwd
!ls ../raw

/global/projecta/projectdirs/desi/external/mws/sv/MWS_CALIB_APOGEE/1/code
apogee_xm_gaia_4DESI_v0.fits


In [20]:
# Parameters

target_class   = 'MWS_CALIB_APOGEE'
input_filename = 'apogee_xm_gaia_4DESI_v0.fits'

REF_CAT        = 'G'
REF_EPOCH      = 2015.5

SELECT_MAG     = 'phot_g_mean_mag'
BRIGHT_LIMIT   = 16.0
FAINT_LIMIT    = 20.0

colmap = {
    'ra':        'RA',
    'dec':       'DEC',
    'pmra':      'PMRA',
    'pmdec':     'PMDEC',
    'source_id': 'REF_ID'
}

{ DESCRIPTION NEEDED }

The selection criteria are:

???

There are some duplicate source IDs in the raw catalog. The corresponding rows are similar (e.g. identical RA, DEC and mag) but some other columns differ. The secondary target indata is restricted to unique `source_id` only.

In [21]:
from   importlib import reload
import sys
PYPATH = '/project/projectdirs/desi/external/mws/py/'
if PYPATH not in sys.path: sys.path.append(PYPATH)
import mwsecondary.mwsecondary as mws
reload(mws)

import os
import sys
import numpy as np

targets = mws.SecondaryTargets(target_class,input_filename,ref_cat=REF_CAT,ref_epoch=REF_EPOCH)
targets.print_info()
mask    = targets.select_finite_mag_range(SELECT_MAG,faint_limit=FAINT_LIMIT,bright_limit=BRIGHT_LIMIT)
mask   &= targets.select_finite_range('dec',lower_limit=-30.0,verbose=True)
mask   &= targets.select_unique('source_id',verbose=True)

# Adjust to secondary target data model and write output
header = dict([(_,globals()[_]) for _ in ['BRIGHT_LIMIT','FAINT_LIMIT']])
output = targets.get_output(mask,cols=colmap,write=True,header=header)
print(output[0:3])

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_CALIB_APOGEE/1/raw/apogee_xm_gaia_4DESI_v0.fits
Read 264032 rows
First 3 rows:
vhelio_avg   verr   vscatter ...       dec             source_id        ebv   
---------- -------- -------- ... ---------------- ------------------- --------
  -98.6882 0.000968      0.0 ... 35.3063874911221 1866727481325405824 0.405349
   14.7794 0.080689 0.337936 ... 35.3474167437634 1866721124773681280 0.351821
  -2.33132 0.008486      0.0 ... 35.3869300615702 1866733700438059776 0.392828

Column names:
['vhelio_avg', 'verr', 'vscatter', 'obsvhelio_avg', 'obsvscatter', 'obsverr', 'synthvhelio_avg', 'synthvscatter', 'synthverr', 'fe_h', 'j', 'h', 'k', 'field', 'location_id', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'pmra', 'pmdec', 'ra', 'dec', 'source_id', 'ebv']


Mag range:  3.69 < phot_g_mean_mag < 21.26
Mags not finite: 0
Selecting 25556 targets from 264032 (9.68%)
Selecting 261879 from 264032 (99.18%) on dec
246444